In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import anndata as ad
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output

import pandas as pd
import warnings
import os
import sys
import time
import gc
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
adata = sc.read('h5ad_files/mouse/ecto_andrea/ecto_nasal_placode_derived_2024.h5ad')

In [ ]:
adata.X = adata.layers['original_counts'].copy()


In [ ]:
adata = adata[adata[:,'Foxe1'].X<1]
adata = adata[adata[:,'Nkx2-3'].X<1]


In [ ]:
adata.X = adata.layers['original_counts'].copy()
sc.pp.normalize_total(adata) # Normalizing to median total counts
sc.pp.log1p(adata) # Logarithmize the data
adata.layers["normalized_counts"] = adata.X.copy()

##highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=500,batch_key = 'sample')

##dimensionality reduction and clustering
sc.tl.pca(adata)
sc.pl.pca_variance_ratio(adata,log = True)


In [ ]:
sc.pp.neighbors(adata,n_pcs = 8)
sc.tl.umap(adata)
sc.tl.leiden(adata,resolution = 1, key_added = 'leiden_post_QC')

sc.pl.umap(adata,color = ['sample','phase','leiden_post_QC',
                         ], ncols = 3, size = 50,legend_loc = 'on data',
           
           cmap = reds, vmin = 0.05)

In [ ]:
# adata = adata[adata.obs['leiden_post_QC'].isin(['0','1','2','3','4','7','8','9','10','12'])]


In [ ]:
# adata.X = adata.layers['original_counts'].copy()
# sc.pp.normalize_total(adata) # Normalizing to median total counts
# sc.pp.log1p(adata) # Logarithmize the data
# adata.layers["normalized_counts"] = adata.X.copy()

# ##highly variable genes
# sc.pp.highly_variable_genes(adata, n_top_genes=300,)

# ##dimensionality reduction and clustering
# sc.tl.pca(adata)
# sc.pl.pca_variance_ratio(adata,log = True)


In [ ]:
# sc.pp.neighbors(adata,n_pcs = 6)
# sc.tl.umap(adata)
# sc.tl.leiden(adata,resolution = 1, key_added = 'leiden_post_QC')

# sc.pl.umap(adata,color = ['sample','phase','leiden_post_QC',
#                          ], ncols = 3, size = 50,legend_loc = 'on data',
           
#            cmap = reds, vmin = 0.05)

In [ ]:
sc.pl.umap(adata,color = ['Xist','Dlx5','Fezf2','Fgf8','Casr','Prnp','Ascl1','Neurog1','Maob','Pcp4','Aldh1a3','Fgf17','Wnt6',
                          'leiden_post_QC',
                         ], ncols = 3, size = 50,legend_loc = 'on data',
           
           cmap = reds, vmin = 0.05)

In [ ]:
sc.tl.rank_genes_groups(adata,groupby='leiden_post_QC',method='wilcoxon')
sc.tl.dendrogram(adata,groupby='leiden_post_QC')
sc.pl.rank_genes_groups_dotplot(adata,
                                #groups = ['2','4','5','6','8','9','10','12','14','15','18',],
                                n_genes = 10, values_to_plot = 'logfoldchanges',
                                cmap='bwr',vmin=-4,vmax=4,min_logfoldchange = 3)

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata,
                                groups = ['9',],
                                n_genes = 30, values_to_plot = 'logfoldchanges',
                                cmap='bwr',vmin=-4,vmax=4,min_logfoldchange = 2)

In [ ]:
sc.pl.umap(adata,color = ['Pitx2','Bmp4','Mgp','Grm7','Sp8','Hes1','Olfm5','Foxa1',
                          'leiden_post_QC',
                         ], ncols = 3, size = 50,legend_loc = 'on data',
           
           cmap = reds, vmin = 0.05)

In [ ]:
scv.pp.filter_genes(adata, min_counts = 5, min_counts_u = 1)
adata = adata[:,(adata.layers['spliced'].sum(axis=0)/adata.layers['unspliced'].sum(axis=0))>.8]

In [ ]:
adata.X = adata.layers['original_counts'].copy()
del adata.uns['log1p']
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=1000)

In [ ]:
scv.pl.proportions(adata,groupby = 'leiden_post_QC',figsize =  (10,8))

In [ ]:
warnings.filterwarnings('ignore')

scv.pp.moments(adata,
               use_highly_variable=True,
               n_neighbors=15)
scv.tl.velocity(adata)
scv.tl.velocity_graph(adata,n_jobs = 10)


In [ ]:
warnings.filterwarnings('ignore')

scv.pl.velocity_embedding_stream(adata, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,)

In [ ]:
scv.pl.velocity(adata, ['Maob','Pcp4','Aldh1a3','Ascl1','Grm7'], color = 'leiden_post_QC',ncols=2)

In [ ]:
scv.tl.rank_velocity_genes(adata, groupby='leiden_post_QC', min_corr=.3)

df = pd.DataFrame(adata.uns['rank_velocity_genes']['names'])

In [ ]:

scv.pl.velocity(adata, df['2'][:10], color = 'leiden_post_QC',ncols=2)

In [ ]:
adata[:,adata.var['highly_variable']==True].var.index

In [ ]:
adata_2023 = sc.read('h5ad_files/mouse/ecto_andrea/ecto_nasal_placode_derived_2023.h5ad')

In [ ]:
adata_2023.var['highly_variable'] = np.where(adata_2023.var.index.isin(list(adata[:,adata.var['highly_variable']==True].var.index)),True,False)

In [ ]:
adata_2023 = adata_2023[:,adata[:,adata.var.index.isin(adata_2023.var.index)].var.index].copy()
adata = adata[:,adata_2023[:,adata_2023.var.index.isin(adata_2023.var.index)].var.index].copy()

In [ ]:
adata_2023.obs['batch']='2023'
adata.obs['batch']='2024'

In [ ]:
adata

In [ ]:
adata_concat = ad.concat([adata,adata_2023],join = 'outer')

In [ ]:
adata_concat.obs['batch'] = adata_concat.obs['batch'].astype('category')
adata_concat.obs['batch']

In [ ]:
adata_concat.var = adata.var

In [ ]:
#adata_concat.write('h5ad_files/mouse/ecto_andrea/ecto_nasal_placode_derived_concat.h5ad')

In [ ]:
adata_concat.X = adata_concat.layers['original_counts'].copy()
sc.pp.normalize_total(adata_concat) # Normalizing to median total counts
sc.pp.log1p(adata_concat) # Logarithmize the data
adata_concat.layers["normalized_counts"] = adata_concat.X.copy()

##highly variable genes
#sc.pp.highly_variable_genes(adata, n_top_genes=500,batch_key = 'sample')

##dimensionality reduction and clustering
sc.tl.pca(adata_concat)
sc.pl.pca_variance_ratio(adata_concat,log = True)


In [ ]:
sc.pp.neighbors(adata_concat,n_pcs = 8)
sc.tl.umap(adata_concat)
sc.tl.leiden(adata_concat,resolution = 1, key_added = 'leiden_post_QC')

sc.pl.umap(adata_concat,color = ['sample','phase','leiden_post_QC','Mecom',
                         ], ncols = 3, size = 50,legend_loc = 'on data',
           
           cmap = reds, vmin = 0.05)

In [ ]:
sc.external.pp.harmony_integrate(adata_concat, key='batch', basis='X_pca', adjusted_basis='X_pca_harmony', )

In [ ]:
sc.pp.neighbors(adata_concat,n_pcs = 12,use_rep = 'X_pca_harmony')
sc.tl.umap(adata_concat)
sc.tl.leiden(adata_concat,resolution = 1, key_added = 'leiden_post_QC')

sc.pl.umap(adata_concat,color = ['sample','phase','leiden_post_QC','Mecom',
                         ], ncols = 3, size = 50,legend_loc = 'on data',
           
           cmap = reds, vmin = 0.05)

In [ ]:
sc.pl.embedding(adata_concat,basis = 'X_pca_harmony', color = 'sample')

In [ ]:
adata_concat

In [ ]:
sc.pl.pca_loadings(adata_concat,components = '1,2')

In [ ]:
adata_int=sc.read('h5ad_files/mouse/ecto_andrea/ecto_nasal_placode_derived_concat_integrated.h5ad')

In [ ]:
image_folder = 'figures_ignore/mouse/mm39/andrea_ecto/'

In [ ]:
genes = ['Casr','Prnp','Neurod1','Neurog1','Ada','Gng8','Fstl5',
 'Gng13','Calb2',
 'S100z','Padi2',
 'Ascl1','Smoc2',
 'Kit','Mybl1', #9
 'Flrt2','Fezf2', #6
 'Acsm4','Pcp4','Maob', #0
 'Wnt4','Six3', #7
 'Dlk1','Sox2ot','Sox6',#12
 'Aldh1a3',#2
 'Trpm3',#3
 'Mgp','Krt14','Foxa1',#15
 'Olfm5',#8
 'Dok5','Bmp4',#13
 'Fxyd3','Pax3',#5
 'Fgf8','Pax7',#10
 'Reg3g','Fmo2',#4
 'Car2','Car4','Sall4','Dlx5',#16
]

In [ ]:
from pathlib import Path
plt.rcParams["figure.figsize"] = [10, 7] 
sc._settings.ScanpyConfig.figdir=Path(image_folder) #set figdir
sc.pl.umap(adata_int,color = genes, ncols = 3, size = 50,legend_loc = 'on data',
           
           cmap = reds, vmin = 0.05,save = '_markers.png')

In [ ]:
plt.rcParams["figure.figsize"] = [10, 7] 

sc.pl.umap(adata_int,color = ['sample',
                              'phase',
                          'leiden',
                         ], ncols = 3, size = 50,
           
           cmap = reds, vmin = 0.05,save = '_dimplot.png')

In [ ]:
sc.tl.rank_genes_groups(adata_int,groupby='leiden',method='wilcoxon')
sc.tl.dendrogram(adata_int,groupby='leiden')
sc.pl.rank_genes_groups_dotplot(adata_int,
                                var_names = genes,
                                #groups = ['13','1',],
                                #n_genes = 10, 
                                values_to_plot = 'logfoldchanges',
                                cmap='bwr',vmin=-4,vmax=4,min_logfoldchange = 2,save = 'markers.png')

In [ ]:
# #DEG between two clusters
# cluster_of_interest = '5'
# reference_cluster = '10'
# cat_name = 'leiden'
# sc.set_figure_params(dpi=50,figsize=[6,4])

# sc.tl.rank_genes_groups(adata_int, cat_name, groups=[cluster_of_interest], reference=reference_cluster, method='wilcoxon')
# df = sc.get.rank_genes_groups_df(adata_int, group=None,)
# df = df[df['logfoldchanges']>2]
# sc.tl.rank_genes_groups(adata_int, groupby = 'leiden', method = 'wilcoxon')
# sc.pl.rank_genes_groups_dotplot(adata_int,var_names = list(df['names'].head(50)),dendrogram=True,
#              values_to_plot="logfoldchanges", cmap='bwr',
#     vmin=-4,
#     vmax=4,)


In [ ]:
adata_int.raw = adata_int.copy()

In [ ]:
scv.pp.filter_genes(adata_int, min_counts = 5, min_counts_u = 1)
adata_int = adata_int[:,(adata_int.layers['spliced'].sum(axis=0)/adata_int.layers['unspliced'].sum(axis=0))>.8]
adata_int = adata_int[:,(adata_int.layers['spliced'].sum(axis=0)/adata_int.layers['unspliced'].sum(axis=0))<30]


In [ ]:
adata_int

In [ ]:
adata_int.X = adata_int.layers['original_counts'].copy()
del adata_int.uns['log1p']
scv.pp.filter_and_normalize(adata_int, min_shared_counts=20, n_top_genes=500)

In [ ]:
scv.set_figure_params(dpi_save = 300)
scv.pl.proportions(adata_int,groupby = 'leiden',figsize =  (10,8),save = image_folder + 'proportions.png')

In [ ]:
warnings.filterwarnings('ignore')

scv.pp.moments(adata_int,
               use_rep = 'X_umap_features',
               #use_highly_variable=True,
               n_neighbors=30)
scv.tl.velocity(adata_int)
scv.tl.velocity_graph(adata_int,n_jobs = 10)


In [ ]:
warnings.filterwarnings('ignore')

scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                save = image_folder + 'velocity_samples.png')

In [ ]:
scv.pl.velocity(adata_int, ['Sult1e1','Maob','Pcp4','Aldh1a3','Ascl1',], color = 'leiden',ncols=2)

In [ ]:
scv.tl.rank_velocity_genes(adata_int, groupby='leiden', min_corr=.3)

df = pd.DataFrame(adata_int.uns['rank_velocity_genes']['names'])

In [ ]:
for i in list(adata_int.obs['leiden'].cat.categories):
    scv.pl.velocity(adata_int, df[i][:10], color = 'leiden',ncols=2,save =image_folder+ i+'_velocity_genes.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='ME8',save = image_folder + 'velocity_samples_ME8.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='ME9',save = image_folder + 'velocity_samples_ME9.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='ME10',save = image_folder + 'velocity_samples_ME10.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='10',save = image_folder + 'velocity_samples_10.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME10','10'],save = image_folder + 'velocity_samples_10-ME10.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['11'],save = image_folder + 'velocity_samples_11.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME11',],save = image_folder + 'velocity_samples_ME11.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME11','11'],save = image_folder + 'velocity_samples_11-ME11.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['12'],save = image_folder + 'velocity_samples_12.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME12',],save = image_folder + 'velocity_samples_ME12.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME12','12'],save = image_folder + 'velocity_samples_12-ME12.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['13'],save = image_folder + 'velocity_samples_13.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['14'],save = image_folder + 'velocity_samples_14.png')


In [ ]:
adata_int=sc.read('h5ad_files/mouse/ecto_andrea/ecto_nasal_placode_derived_concat_integrated_nocycle.h5ad')

In [ ]:
image_folder = 'figures_ignore/mouse/mm39/andrea_ecto/'

In [ ]:
genes = ['Casr','Prnp','Neurod1','Neurog1','Ada','Gng8','Fstl5',
 'Gng13','Calb2',
 'S100z','Padi2',
 'Ascl1','Smoc2',
 'Kit','Mybl1', #9
 'Flrt2','Fezf2', #6
 'Acsm4','Pcp4','Maob', #0
 'Wnt4','Six3', #7
 'Dlk1','Sox2ot','Sox6',#12
 'Aldh1a3',#2
 'Trpm3',#3
 'Mgp','Krt14','Foxa1',#15
 'Olfm5',#8
 'Dok5','Bmp4',#13
 'Fxyd3','Pax3',#5
 'Fgf8','Pax7',#10
 'Reg3g','Fmo2',#4
 'Car2','Car4','Sall4','Dlx5',#16
         'Plac9','Ppic','Aqp3','Hmcn1','Csmd2','Ptprn2','Stum','Rasgrp1','Tmprss4','Fgf3',
]

In [ ]:
from pathlib import Path
plt.rcParams["figure.figsize"] = [10, 7] 
sc._settings.ScanpyConfig.figdir=Path(image_folder) #set figdir
sc.pl.umap(adata_int,color = genes, ncols = 3, size = 50,legend_loc = 'on data',
           
           cmap = reds, vmin = 0.05,
           save = '_markers.png'
          )

In [ ]:
plt.rcParams["figure.figsize"] = [10, 7] 

sc.pl.umap(adata_int,color = ['sample',
                              'phase',
                          'leiden',
                         ], ncols = 3, size = 50,
           
           cmap = reds, vmin = 0.05,
           save = '_dimplot.png'
          )

In [ ]:
sc.tl.rank_genes_groups(adata_int,groupby='leiden',method='wilcoxon')
sc.tl.dendrogram(adata_int,groupby='leiden')
sc.pl.rank_genes_groups_dotplot(adata_int,
                                var_names = genes,
                                #groups = ['13','1',],
                                #n_genes = 10, 
                                values_to_plot = 'logfoldchanges',
                                cmap='bwr',vmin=-4,vmax=4,min_logfoldchange = 2,
                                save = 'markers.png'
                               )

In [ ]:
# #DEG between two clusters
# cluster_of_interest = '5'
# reference_cluster = '10'
# cat_name = 'leiden'
# sc.set_figure_params(dpi=50,figsize=[6,4])

# sc.tl.rank_genes_groups(adata_int, cat_name, groups=[cluster_of_interest], reference=reference_cluster, method='wilcoxon')
# df = sc.get.rank_genes_groups_df(adata_int, group=None,)
# df = df[df['logfoldchanges']>2]
# sc.tl.rank_genes_groups(adata_int, groupby = 'leiden', method = 'wilcoxon')
# sc.pl.rank_genes_groups_dotplot(adata_int,var_names = list(df['names'].head(50)),dendrogram=True,
#              values_to_plot="logfoldchanges", cmap='bwr',
#     vmin=-4,
#     vmax=4,)


## Notes
The 2024 data was subset in the ectoderm_spinoff.ipynb notebook. First Epcam positive cells were subset. Subsequently the purported placodal derived cells were subset.  
The 2023 ectoderm was loaded as raw cells and quickly clustered. Placode derived cells were subset based on Sp8 and Six1 expression in the ectoderm_subset_2023.ipynb notebook.  
Data was loaded in this notebook and combined. Highly variable genes from the 2023 data were kept.  
In the placodes_2024_mira.ipynb notebook cell cycle associated genes were removed from the highly variable genes before integration with MIRA.  
Integration was done with 8 topics and 100 epochs. Data was then exported again to this notebook for further analysis.  
For velocity analysis, cell cycle associated genes were removed. Genes with abnormal spliced/unspliced ratios were removed.  
min_counts = 5, min_counts_u = 1 and min_shared_counts=20  
scv.pp.filter_and_normalize was used to select the top 300 genes for velocity analysis.  
Velocity was computed with 30 nearest neighbours, using the X_umap_features from MIRA as a basis.  

Small note, I see some Oc90+ cells, this must be inner ear. They are from the 2023 data. Clearly we missed some when filtering last year.  
I'm not sure how worth it is to go back to this data again.

In [ ]:
adata_int=sc.read('h5ad_files/mouse/ecto_andrea/ecto_nasal_placode_derived_concat_integrated_nocycle.h5ad') #8 TOPICS 100 EPOCHS

In [ ]:
adata_int.raw = adata_int.copy()

In [ ]:
adata_int = adata_int[:,adata_int.var['S_score']<.1].copy()
adata_int = adata_int[:,adata_int.var['G2M_score']<.1].copy()

In [ ]:
scv.pp.filter_genes(adata_int, min_counts = 5, min_counts_u = 1)
adata_int = adata_int[:,(adata_int.layers['spliced'].sum(axis=0)/adata_int.layers['unspliced'].sum(axis=0))>.8]
adata_int = adata_int[:,(adata_int.layers['spliced'].sum(axis=0)/adata_int.layers['unspliced'].sum(axis=0))<30]


In [ ]:
adata_int.X = adata_int.layers['original_counts'].copy()
del adata_int.uns['log1p']
scv.pp.filter_and_normalize(adata_int, min_shared_counts=20, n_top_genes=300)

In [ ]:
scv.set_figure_params(dpi_save = 300)
scv.pl.proportions(adata_int,groupby = 'leiden',figsize =  (10,8),
                   save = image_folder + 'proportions.png'
                  )

In [ ]:
warnings.filterwarnings('ignore')

scv.pp.moments(adata_int,
               use_rep = 'X_umap_features',
               #use_highly_variable=True,
               n_neighbors=30)
scv.tl.velocity(adata_int)
scv.tl.velocity_graph(adata_int,n_jobs = 10)


In [ ]:
warnings.filterwarnings('ignore')

scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                save = image_folder + 'velocity_samples.png'
                                )

In [ ]:
warnings.filterwarnings('ignore')

scv.pl.velocity_embedding_stream(adata_int, color = 'leiden',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                save = image_folder + 'velocity_leiden.png'
                                )

In [ ]:
scv.tl.rank_velocity_genes(adata_int, groupby='leiden', min_corr=.3)

df = pd.DataFrame(adata_int.uns['rank_velocity_genes']['names'])

In [ ]:
for i in list(adata_int.obs['leiden'].cat.categories):
    scv.pl.velocity(adata_int, df[i][:10], color = 'leiden',ncols=2,save =image_folder+ i+'_velocity_genes.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='ME8',save = image_folder + 'velocity_samples_ME8.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='ME9',save = image_folder + 'velocity_samples_ME9.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='ME10',save = image_folder + 'velocity_samples_ME10.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups='10',save = image_folder + 'velocity_samples_10.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME10','10'],save = image_folder + 'velocity_samples_10-ME10.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['11'],save = image_folder + 'velocity_samples_11.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME11',],save = image_folder + 'velocity_samples_ME11.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME11','11'],save = image_folder + 'velocity_samples_11-ME11.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['12'],save = image_folder + 'velocity_samples_12.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME12',],save = image_folder + 'velocity_samples_ME12.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['ME12','12'],save = image_folder + 'velocity_samples_12-ME12.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['13'],save = image_folder + 'velocity_samples_13.png')
scv.pl.velocity_embedding_stream(adata_int, color = 'sample',min_mass = 0,density=2.5,legend_loc = 'right margin',size=13, alpha=1,
                                 groups=['14'],save = image_folder + 'velocity_samples_14.png')
